## Modeling ##

In [1]:
import time
import pickle
import os
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib.colors import Colormap
import scikitplot as skplt
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, SimpleImputer, IterativeImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.metrics import (
    classification_report, 
    confusion_matrix,
    r2_score,
    mean_squared_error, 
    root_mean_squared_error,
    mean_absolute_error, 
    mean_absolute_percentage_error,
    accuracy_score,
    matthews_corrcoef,
    brier_score_loss,
    f1_score
)
from sklearn.inspection import PartialDependenceDisplay
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors

from sklearn.linear_model import LogisticRegression

from faiss_imputer import FaissImputer

from xgboost import XGBClassifier

from sklearn.neural_network import MLPClassifier

from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
os.chdir('..')
print(f'Current working directory is {os.getcwd()}')

This notebook is dedicated to the feature selection and statistical modeling of our trucking data.

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('../data/data_clean_05_05.csv', low_memory=False)

Cleaning features and reassigning them to the proper dtypes

In [4]:
columns_to_object = ['ecuSource',
                     'spn',
                     'fmi',
                     'MCTNumber',
                     'RecordID',
                     'ESS_Id'
                    ]

for column in columns_to_object:
    df[column] = df[column].astype(object)

In [5]:
columns_to_bool = ['CruiseControlActive',
                   'IgnStatus',
                   'ParkingBrake']

for column in columns_to_bool:
    df[column] = df[column].astype(bool)

In [6]:
int64_cols = df.select_dtypes(include='bool').columns
df[int64_cols] = df[int64_cols].astype('int64')

Separating the test and training data.

In [7]:
test_date = '2019-01-01'

df_test = df.sort_values('EventTimeStamp').loc[df['EventTimeStamp'] > test_date]

In [8]:
df_train = df.sort_values('EventTimeStamp').loc[df['EventTimeStamp'] < test_date]

In [9]:
#df = df.sample(frac=0.50)

Scaling and encoding features for modeling.

In [14]:
X_train = df_train.drop(columns = [
            'target',  
            'LocationTimeStamp',
            'EventTimeStamp',
            'eventDescription',
            'ecuSerialNumber',
            'ecuSoftwareVersion',
            'time_derate',
            'time_until_derate',
            'Longitude',
            'Latitude',
            'ESS_Id',
            'RecordID',
            'ecuModel',
            'ServiceDistance',
            'ecuMake',
            'SwitchedBatteryVoltage',
            'MCTNumber',
            'EquipmentID',
            'LampStatus',
            'CruiseControlSetSpeed',
            'EngineLoad',
            'TurboBoostPressure',
            'DistanceLtd'
            ], axis=1)

y_train = df_train['target']

In [15]:
X_test = df_test.drop(columns = [
            'target',  
            'LocationTimeStamp',
            'EventTimeStamp',
            'eventDescription',
            'ecuSerialNumber',
            'ecuSoftwareVersion',
            'time_derate',
            'time_until_derate',
            'Longitude',
            'Latitude',
            'ESS_Id',
            'RecordID',
            'ecuModel',
            'ServiceDistance',
            'ecuMake',
            'SwitchedBatteryVoltage',
            'MCTNumber',
            'EquipmentID',
            'LampStatus',
            'CruiseControlSetSpeed',
            'EngineLoad',
            'TurboBoostPressure',
            'DistanceLtd'
            ], axis=1)

y_test = df_test['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=27, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify = y_train, random_state = 27, train_size = 0.6/0.8)

Selection of features for each step of the pipeline. The last few lines are for checking to make sure each feature is accounted for.

In [25]:
ohe_features = ['spn',
                'fmi',
                'ecuSource'
                ] 

bool_features = ['CruiseControlActive',
                 'IgnStatus',
                 'ParkingBrake',
                 'active'
                ]

scale_features = ['AcceleratorPedal',
                  'BarometricPressure',
                  'EngineCoolantTemperature',
                  'EngineOilPressure',
                  'EngineOilTemperature',
                  'EngineRpm',
                  'FuelLevel',
                  'FuelLtd',
                  'FuelTemperature',
                  'IntakeManifoldTemperature',
                  'Speed',
                  'FuelRate',
                  'EngineTimeLtd',
                  'Throttle',
                  'activeTransitionCount'
                 ]

everything = list(set(ohe_features + bool_features + scale_features))
the_rest = X_train.columns.difference(everything)
pca = PCA(n_components = 4)

In [42]:
%%time

numerical_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('ii', IterativeImputer(initial_strategy = 'mean',
                           max_iter = 100))
])

categorical_pipeline = Pipeline([
    ('ohe', OneHotEncoder(categories='auto', 
                          handle_unknown = 'ignore')),
    ('si', SimpleImputer(strategy='most_frequent'))
])

boolean_pipeline = Pipeline([
    ('si', SimpleImputer(strategy='most_frequent'))
])

ct = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, scale_features),
        ('cat', categorical_pipeline, ohe_features),
        ('bool', boolean_pipeline, bool_features)
    ],
    remainder='drop'
)

pipe = Pipeline(
    steps=[
        ('transformer', ct)
    ])

pipe.fit(X_train)

C:\Users\cavin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


CPU times: total: 5min 20s
Wall time: 2min 52s


Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler()),
                                                                  ('ii',
                                                                   IterativeImputer())]),
                                                  ['AcceleratorPedal',
                                                   'BarometricPressure',
                                                   'DistanceLtd',
                                                   'EngineCoolantTemperature',
                                                   'EngineLoad',
                                                   'EngineOilPressure',
                                                   'EngineOilTemperature',
                                                   'EngineRpm', 'FuelLevel',
                                                   'FuelLtd', 'FuelTemperature',
                                                   'IntakeManifoldTem...
                                                   'FuelRate', 'EngineTimeLtd',
                                                   'CruiseControlSetSpeed',
                                                   'Throttle',
                                                   'activeTransitionCount']),
                                                 ('cat',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('si',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['spn', 'fmi', 'ecuSource']),
                                                 ('bool',
                                                  Pipeline(steps=[('si',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['CruiseControlActive',
                                                   'IgnStatus', 'ParkingBrake',
                                                   'active'])]))])

The above cell takes 5 minutes to fit the pipe, and the cell below takes 1 minute to transform the pipe.

In [43]:
%%time

X_train_transformed = pipe.transform(X_train)
X_test_transformed = pipe.transform(X_test) 

CPU times: total: 45.2 s
Wall time: 34.4 s


Transformation of pipe and saving the pipe as a pickle object so that the pipe doesn't need to be fitted again.

In [44]:
%%time

filename = 'pipe_transformed.pkl'

pickle_list = [pipe, X_train_transformed, X_test_transformed]

with open(filename, 'wb') as file:
    pickle.dump(pickle_list, file)

CPU times: total: 469 ms
Wall time: 610 ms


In [45]:
%%time

filename = 'pipe_transformed.pkl'

with open(filename, 'rb') as file:
    pipe, X_train_transformed, X_test_transformed = pickle.load(file)

CPU times: total: 203 ms
Wall time: 232 ms


Applying the pipe transformations to models to see which model performs best.

In [46]:
%%time

knn_model = KNeighborsClassifier().fit(X_train_transformed, y_train)

CPU times: total: 234 ms
Wall time: 243 ms


In [ ]:
%%time

y_pred_knn = knn_model.predict(X_test_transformed)

Prediction on KNeighborsClassifier takes a while

In [ ]:
print(f'Accuracy: {accuracy_score(y_test, y_pred_knn)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred_knn)}')
print(confusion_matrix(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn, zero_division = 0))
#print(cross_val_score(knn_model, X_train_transformed, y_train, cv=3))

In [37]:
%%time

xgb = XGBClassifier().fit(X_train_transformed, y_train)

y_pred_xgb = xgb.predict(X_test_transformed)

CPU times: total: 29.5 s
Wall time: 3.04 s


In [38]:
print(f'Accuracy: {accuracy_score(y_test, y_pred_xgb)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred_xgb)}')
print(confusion_matrix(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb, zero_division = 0))

Accuracy: 0.9979370532150578
MCC: -0.0001917304221267552
[[111261      2]
 [   228      0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    111263
           1       0.00      0.00      0.00       228

    accuracy                           1.00    111491
   macro avg       0.50      0.50      0.50    111491
weighted avg       1.00      1.00      1.00    111491



In [ ]:
explainer = shap.TreeExplainer(xgb)
explanation = explainer(X_test_transformed)

In [ ]:
shap.plots.beeswarm(explanation)

In [ ]:
%%time

rfc = RandomForestClassifier(n_estimators=5, max_depth=5, random_state=27).fit(x_train_transformed, y_train)

y_pred_rfc = rfc.predict(X_test_transformed)

In [ ]:
print(f'Accuracy: {accuracy_score(y_test, y_pred_rfc)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred_rfc)}')
print(confusion_matrix(y_test, y_pred_rfc))
print(classification_report(y_test, y_pred_rfc, zero_division = 0))

In [ ]:
from sklearn.inspection import permutation_importance

start_time = time.time()
result = permutation_importance(
    rfc, X_test_transformed, y_test, n_repeats=10, random_state=42, n_jobs=2
)
elapsed_time = time.time() - start_time
print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

forest_importances = pd.Series(result.importances_mean, index=feature_names)

In [ ]:
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=result.importances_std, ax=ax)
ax.set_title("Feature importances using permutation on full model")
ax.set_ylabel("Mean accuracy decrease")
fig.tight_layout()
plt.show()

In [ ]:
skplt.metrics.plot_confusion_matrix(y_test, y_pred_rfc, normalize=True)
plt.show()

In [ ]:
%%time

logreg = LogisticRegression(max_iter=10000).fit(X_train_transformed, y_train)

y_pred_logreg = logreg.predict(X_test_transformed)

In [ ]:
print(f'Accuracy: {accuracy_score(y_test, y_pred_logreg)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred_logreg)}')
print(confusion_matrix(y_test, y_pred_logreg))
print(classification_report(y_test, y_pred_logreg, zero_division = 0))

y_val_pred_proba = pipe.predict_proba(X_val)[:,1]

candidate_thresholds = np.arange(start = 0.1, stop = 0.925, step = 0.01)
thresholds = pd.DataFrame({'threshold': candidate_thresholds})
thresholds['f1'] = thresholds['threshold'].apply(lambda x: f1_score(y_val, y_val_pred_proba > x))
thresholds.sort_values('f1', ascending = False).head()

threshold = 0.10

y_pred_proba = model.predict_proba(X_test_transformed)[:,1]

y_pred = y_pred_proba > threshold
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
#param_grid = {
#    'preprocessor__num__scaler__with_mean': [True, False],
#    'preprocessor__num__scaler__with_std': [True, False],
#    'classifier__C': [0.1, 1, 10],
#    'classifier__solver': ['liblinear', 'newton-cg']
#}

#randomized_search = RandomizedSearchCV(pipeline, param_grid, n_iter=10, cv=3)

In [ ]:
%%time

hgbc = HistGradientBoostingClassifier().fit(X_train_transformed, y_train)

y_pred_hgbc = hgbc.predict(X_test_transformed)

In [ ]:
confusion_matrix(y_test, y_pred_hgbc)

In [ ]:
print(classification_report(y_test, y_pred_hgbc, zero_division = 0))

In [ ]:
dtc = DecisionTreeClassifier().fit(X_train_transformed, y_train)

y_pred_dtc = dtc.predict(X_test_transformed)

In [ ]:
print(f'Accuracy: {accuracy_score(y_test, y_pred_dtc)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred_dtc)}')
print(confusion_matrix(y_test, y_pred_dtc))
print(classification_report(y_test, y_pred_dtc, zero_division = 0))

In [ ]:
rf = RandomForestClassifier()
lr = LogisticRegression()
dtc = DecisionTreeClassifier()
xgb = XGBClassifier()
rf_probas = rf.fit(X_train_transformed, y_train).predict_proba(X_test_transformed)
lr_probas = lr.fit(X_train_transformed, y_train).predict_proba(X_test_transformed)
dtc_probas = dtc.fit(X_train_transformed, y_train).predict_proba(X_test_transformed)
xgb_scores = xgb.fit(X_train_transformed, y_train).predict_proba(X_test_transformed)
probas_list = [rf_probas, lr_probas, dtc_probas, xgb_scores]
clf_names = ['Random Forest', 'Logistic Regression',
           'Decision Tree', 'XGBoost']
skplt.metrics.plot_calibration_curve(y_test,
                                     probas_list,
                                     clf_names)
plt.show()